In [8]:
import os
import pickle
import torch
import numpy as np
from math import ceil
from model_vc import Generator

metadata = pickle.load(open('/home/ytang363/7100_voiceConversion/autovc-master/metadata.pkl', "rb"))
train = pickle.load(open('/home/ytang363/7100_voiceConversion/VCTK-Corpus-0.92/spmel_test/train.pkl', "rb"))

In [20]:
for sbmt_i in metadata:
    print(sbmt_i[2].shape)

(90, 80)
(89, 80)
(75, 80)
(109, 80)


In [14]:
for sbmt_i in train:
    print(sbmt_i)

['p225', array([ 4.80997525e-02, -3.97536606e-02, -5.97422309e-02,  4.44349013e-02,
       -2.26828530e-02,  1.04555404e-02, -2.89933234e-02,  5.96510665e-03,
       -8.52530971e-02, -2.75070406e-02,  5.69717102e-02,  1.29156187e-03,
        5.22593781e-02,  4.97497134e-02,  7.52646476e-04, -4.43350673e-02,
       -1.69802699e-02,  7.92745054e-02,  1.27056390e-02, -3.80487996e-03,
        1.93539746e-02, -4.40648906e-02,  5.29617369e-02,  5.18266782e-02,
        1.98383667e-02,  5.67989945e-02,  2.04613172e-02, -3.47547606e-02,
        9.06099752e-03, -1.75395254e-02,  4.37302217e-02,  2.01737061e-02,
       -1.61580443e-02, -4.17732373e-02,  1.11317951e-02, -4.39894125e-02,
       -2.60744877e-02, -4.89442609e-02,  1.29641024e-02,  1.02108857e-02,
        4.78984267e-02,  3.10389213e-02,  5.11142090e-02, -3.34549472e-02,
       -1.75584033e-02,  1.04206037e-02, -4.57631163e-02,  7.69352838e-02,
       -5.50725535e-02,  7.22244848e-03, -4.00303528e-02,  1.45068225e-02,
       -2.305663